# Pandas practice (part 3)

In [2]:
import pandas as pd
import numpy as np

## 1. Dummy variables
- Là biến nhị phân (chỉ có giá trị 0 hoặc 1)
- One-hot encoding

In [146]:
# Create fake data
df = pd.DataFrame({
    'id': range(1, 5),
    'sex': ['male', 'male', 'female', 'male'],
    'age': [18, 19, 22, 15]

})

df

,id,sex,age
0,1,male,18
1,2,male,19
2,3,female,22
3,4,male,15


In [5]:
# Get dummies
pd.get_dummies(df["sex"])

,female,male
0,0,1
1,0,1
2,1,0
3,0,1


In [6]:
# Get dummies with prefix
pd.get_dummies(df["sex"], prefix="is")

,is_female,is_male
0,0,1
1,0,1
2,1,0
3,0,1


In [147]:
# Get dummies drop first
sex_dummies = pd.get_dummies(df["sex"], prefix="is", drop_first=True)
sex_dummies

,is_male
0,1
1,1
2,0
3,1


In [149]:
pd.concat([df, sex_dummies], axis=1)

,id,sex,age,is_male
0,1,male,18,1
1,2,male,19,1
2,3,female,22,0
3,4,male,15,1


In [15]:
df = df.join(sex_dummies).drop("sex", axis=1)

In [16]:
df

,id,age,is_male
0,1,18,1
1,2,19,1
2,3,22,0
3,4,15,1


Example with 4 classes

In [17]:
# Create fake data
df = pd.DataFrame({
    'id': range(1, 8),
    'sex': ['male', 'male', 'female', 'male', 'male', 'female', 'male'],
    'ranking': ["Excellent", "Excellent", "Good", "Avg", "Avg", "Failed", "Failed"]

})

df

,id,sex,ranking
0,1,male,Excellent
1,2,male,Excellent
2,3,female,Good
3,4,male,Avg
4,5,male,Avg
5,6,female,Failed
6,7,male,Failed


In [19]:
sex_dummies = pd.get_dummies(df["sex"], prefix="is", drop_first=True)
sex_dummies.head(3)

,is_male
0,1
1,1
2,0


In [21]:
ranking_dummies = pd.get_dummies(df["ranking"], prefix="is", drop_first=True)
ranking_dummies.head()

,is_Excellent,is_Failed,is_Good
0,1,0,0
1,1,0,0
2,0,0,1
3,0,0,0
4,0,0,0


In [30]:
df\
    .join(sex_dummies)\
    .join(ranking_dummies)\
    .drop(columns=["sex", "ranking"])

,id,is_male,is_Excellent,is_Failed,is_Good
0,1,1,1,0,0
1,2,1,1,0,0
2,3,0,0,0,1
3,4,1,0,0,0
4,5,1,0,0,0
5,6,0,0,1,0
6,7,1,0,1,0


## 2. Handle strings

In [56]:
# Create fake data (notice spaces at the end end beginning)
data = {
    "name": [" Bob J. Dylan", " Jane Junior Adam Smith ", "Steve Jefferson ", "Jack Robinson", "Trevor Noah"],
    "email": ["bob@gmail.com", "jane@gmail.com", "steve@hotmail.com", "jack@yahoo.com", np.nan]
}

df = pd.DataFrame(data)
df

,name,email
0,Bob J. Dylan,bob@gmail.com
1,Jane Junior Adam Smith,jane@gmail.com
2,Steve Jefferson,steve@hotmail.com
3,Jack Robinson,jack@yahoo.com
4,Trevor Noah,NaN


Find users with gmail

In [38]:
# Masking
df["email"].str.contains("gmail") == True

0     True
1     True
2    False
3    False
4    False
Name: email, dtype: bool

In [40]:
# Lọc ra gmail
df.loc[df["email"].str.contains("gmail") == True, :]

,name,email
0,Bob Dylan,bob@gmail.com
1,Jane Smith,jane@gmail.com


In [41]:
# Uppercase name
df["name"].str.upper()

0           BOB DYLAN
1         JANE SMITH 
2    STEVE JEFFERSON 
3       JACK ROBINSON
4         TREVOR NOAH
Name: name, dtype: object

In [42]:
# Lowercase name
df["name"].str.lower()

0           bob dylan
1         jane smith 
2    steve jefferson 
3       jack robinson
4         trevor noah
Name: name, dtype: object

In [44]:
# Trim space
df["name"].str.strip()

0          Bob Dylan
1         Jane Smith
2    Steve Jefferson
3      Jack Robinson
4        Trevor Noah
Name: name, dtype: object

Get first name, last name using `apply` and `split`

In [45]:
df

,name,email
0,Bob Dylan,bob@gmail.com
1,Jane Smith,jane@gmail.com
2,Steve Jefferson,steve@hotmail.com
3,Jack Robinson,jack@yahoo.com
4,Trevor Noah,NaN


In [51]:
# Test split
df["name"].str.split()

0          [Bob, Dylan]
1         [Jane, Smith]
2    [Steve, Jefferson]
3      [Jack, Robinson]
4        [Trevor, Noah]
Name: name, dtype: object

In [57]:
df

,name,email
0,Bob J. Dylan,bob@gmail.com
1,Jane Junior Adam Smith,jane@gmail.com
2,Steve Jefferson,steve@hotmail.com
3,Jack Robinson,jack@yahoo.com
4,Trevor Noah,NaN


In [58]:
df

,name,email
0,Bob J. Dylan,bob@gmail.com
1,Jane Junior Adam Smith,jane@gmail.com
2,Steve Jefferson,steve@hotmail.com
3,Jack Robinson,jack@yahoo.com
4,Trevor Noah,NaN


In [84]:
def get_names(x):
    words = x.split()
    
    # Logic
    if len(words) > 2:
        first_name = words.pop(0)
        last_name = words.pop()
        middle_name = " ".join(words)
    else:
        first_name = words[0]
        middle_name = ""
        last_name = words[-1]
        
    return (first_name, middle_name, last_name)

In [88]:
df["first"] = df["name"].apply(lambda x: get_names(x)[0])
df["middle"] = df["name"].apply(lambda x: get_names(x)[1])
df["last"] = df["name"].apply(lambda x: get_names(x)[-1])

In [89]:
df

,name,email,first,middle,last
0,Bob J. Dylan,bob@gmail.com,Bob,J.,Dylan
1,Jane Junior Adam Smith,jane@gmail.com,Jane,Junior Adam,Smith
2,Steve Jefferson,steve@hotmail.com,Steve,,Jefferson
3,Jack Robinson,jack@yahoo.com,Jack,,Robinson
4,Trevor Noah,NaN,Trevor,,Noah


In [25]:
# Use apply and split to get first name
df["name"].apply(lambda x: x.split()[0])

0       Bob
1      Jane
2     Steve
3      Jack
4    Trevor
Name: name, dtype: object

In [26]:
# Complete code
df["first_name"] = df["name"].apply(lambda x: x.split()[0])
df["last_name"] = df["name"].apply(lambda x: x.split()[-1])
df

,name,email,first_name,last_name
0,Bob Dylan,bob@gmail.com,Bob,Dylan
1,Jane Smith,jane@gmail.com,Jane,Smith
2,Steve Jefferson,steve@hotmail.com,Steve,Jefferson
3,Jack Robinson,jack@yahoo.com,Jack,Robinson
4,Trevor Noah,NaN,Trevor,Noah


## 4. Multi level index

In [90]:
# Create fake data
index = [
    ['US', 'US', 'US', 'CA', 'CA', 'CA', 'CN', 'CN', 'CN'],
    ["gdp", "pop", "area"] * 3
]

s = pd.Series(np.random.randn(9),index=index)
s

US  gdp    -0.041549
    pop     2.503813
    area    1.315864
CA  gdp    -0.114200
    pop     1.721585
    area    0.529671
CN  gdp    -0.305723
    pop     1.288061
    area   -0.661753
dtype: float64

### Filter

In [91]:
s.loc["US"]

gdp    -0.041549
pop     2.503813
area    1.315864
dtype: float64

In [94]:
s["US"]

gdp    -0.041549
pop     2.503813
area    1.315864
dtype: float64

In [95]:
s["US", "gdp"]

-0.04154872196699843

In [104]:
s[["US", "CA"]][:, "gdp"]

US   -0.041549
CA   -0.114200
dtype: float64

In [107]:
# Level 2
s[:, "gdp"]

US   -0.041549
CA   -0.114200
CN   -0.305723
dtype: float64

### Stack/Unstack

In [108]:
s

US  gdp    -0.041549
    pop     2.503813
    area    1.315864
CA  gdp    -0.114200
    pop     1.721585
    area    0.529671
CN  gdp    -0.305723
    pop     1.288061
    area   -0.661753
dtype: float64

In [109]:
# Unstack
s.unstack()

,area,gdp,pop
CA,0.529671,-0.114200,1.721585
CN,-0.661753,-0.305723,1.288061
US,1.315864,-0.041549,2.503813


In [110]:
# Stack
s.unstack().stack()

CA  area    0.529671
    gdp    -0.114200
    pop     1.721585
CN  area   -0.661753
    gdp    -0.305723
    pop     1.288061
US  area    1.315864
    gdp    -0.041549
    pop     2.503813
dtype: float64

### Summary statistics by level

In [111]:
# Create fake data
data = {
    "month_1": np.random.randint(1, 11, 9),
    "month_2": np.random.randint(1, 11, 9)
}

index = [
    ['US', 'US', 'US', 'CA', 'CA', 'CA', 'CN', 'CN', 'CN'],
    ["gdp", "pop", "area"] * 3
]

df = pd.DataFrame(data, index=index)
df

month_1  month_2
US gdp         1        7
   pop         6       10
   area        1        5
CA gdp         5        5
   pop         8        7
   area        1       10
CN gdp         3        3
   pop         8        5
   area        4        2

In [112]:
# Default
df.sum()

month_1    37
month_2    54
dtype: int64

In [114]:
# Level 0
df.sum(level=0)

,month_1,month_2
US,8,22
CA,14,22
CN,15,10


In [115]:
# Level 1
df.sum(level=1)

,month_1,month_2
gdp,9,15
pop,22,22
area,6,17


Similar for other summary: mean, std, cumsum, ... (aggregation)

### Set index

In [116]:
data = {
    'a': range(7), 
    'b': range(7, 0, -1),
    'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
    'd': [0, 1, 2, 0, 1, 2, 3]
}

df = pd.DataFrame(data)
df

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [117]:
df.set_index(["c", "d"])

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [118]:
# Set index
df2 = df.set_index(["c", "d"])
df2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [121]:
# Reset index
df2.reset_index(drop=True)

,a,b
0,0,7
1,1,6
2,2,5
3,3,4
4,4,3
5,5,2
6,6,1


## 5. Join DFs

In [122]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})

In [123]:
display(df1)
display(df2)

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,key,data2
0,a,0
1,b,1
2,d,2


### Demo
![](https://d33wubrfki0l68.cloudfront.net/108c0749d084c03103f8e1e8276c20e06357b124/5f113/diagrams/join-setup.png)

![](https://d33wubrfki0l68.cloudfront.net/820b012580731f2134f90ee9c6388994c2343683/27703/diagrams/join-setup2.png)

![](https://d33wubrfki0l68.cloudfront.net/3abea0b730526c3f053a3838953c35a0ccbe8980/7f29b/diagrams/join-inner.png)

#### Inner
![](https://d33wubrfki0l68.cloudfront.net/3abea0b730526c3f053a3838953c35a0ccbe8980/7f29b/diagrams/join-inner.png)

#### Outer 
![](https://d33wubrfki0l68.cloudfront.net/9c12ca9e12ed26a7c5d2aa08e36d2ac4fb593f1e/79980/diagrams/join-outer.png)

### Ven diagrams
![](https://d33wubrfki0l68.cloudfront.net/aeab386461820b029b7e7606ccff1286f623bae1/ef0d4/diagrams/join-venn.png)

### Inner join
- Chỉ giữ lại những dòng mà values xuất hiện trên cả 2 bảng.

In [124]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [125]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [126]:
pd.merge(df1, df2, on="key", how="inner")

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


### Left (outer) join
- Giữ lại tất cả những dòng của bảng left, những dòng mà bảng right ko có key values tương ứng thì fill `NaN`.
- Tương tự cho `how="right"`.

In [127]:
pd.merge(df1, df2, on=["key"], how="left")

,key,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


In [129]:
pd.merge(df2, df1, on=["key"], how="left")

,key,data2,data1
0,a,0,2.0
1,a,0,4.0
2,a,0,5.0
3,b,1,0.0
4,b,1,1.0
5,b,1,6.0
6,d,2,NaN


### Full (outer) join
- Giữ lại tất cả các dòng có key xuất hiện ở ít nhất một trong hai bảng.
- Bảng nào không có thì fill `NaN`.

In [130]:
pd.merge(df1, df2, on=["key"], how="outer")

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


### 4.4. Anti join
- Anti join là giữ lại những dòng có key values nằm trong bảng 1 nhưng không nằm trong bảng 2.
- Pandas không hỗ trợ anti join.
- Muốn dùng thì phải left join sau đó filter.

In [132]:
pd.merge(df1, df2, on=["key"], how="left")

,key,data1,data2
0,b,0,1.0
1,b,1,1.0
2,a,2,0.0
3,c,3,NaN
4,a,4,0.0
5,a,5,0.0
6,b,6,1.0


In [134]:
df2["dummy"] = 1
df3 = pd.merge(df1, df2, on=["key"], how="left")
df3

,key,data1,data2,dummy
0,b,0,1.0,1.0
1,b,1,1.0,1.0
2,a,2,0.0,1.0
3,c,3,NaN,NaN
4,a,4,0.0,1.0
5,a,5,0.0,1.0
6,b,6,1.0,1.0


In [135]:
df2["dummy"] = 1

df3 = pd.merge(df1, df2, on=["key"], how="left")
df3 = df3.loc[df3["dummy"].isnull() == True, :].drop(["dummy"], axis=1)
df3

,key,data1,data2
3,c,3,NaN


## 6. Concat DFs

In [136]:
# Create fake data
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

display(df1)
display(df2)

,a,b,c,d
0,-0.448295,-0.055613,0.273334,-0.541111
1,1.316187,-0.364378,-0.047264,-0.071256
2,-2.327875,0.353559,-0.185406,0.889948


,b,d,a
0,-0.585175,0.079060,-0.526320
1,0.331067,-0.312151,-1.214595


In [143]:
# Row-wise
df = pd.concat([df1, df2], sort=True, ignore_index=True)
df

,a,b,c,d
0,-0.448295,-0.055613,0.273334,-0.541111
1,1.316187,-0.364378,-0.047264,-0.071256
2,-2.327875,0.353559,-0.185406,0.889948
3,-0.526320,-0.585175,NaN,0.079060
4,-1.214595,0.331067,NaN,-0.312151


In [144]:
# Create fake data
df1 = pd.DataFrame(np.random.randn(3, 2), columns=['X', 'Y'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

display(df1)
display(df2)

,X,Y
0,-2.048713,1.073073
1,-0.332568,-1.161011
2,0.553347,0.346223


,b,d,a
0,-0.034724,-0.512885,0.319018
1,0.890191,0.944020,0.801395


In [145]:
# Column-wise
pd.concat([df1, df2], axis=1) # 

,X,Y,b,d,a
0,-2.048713,1.073073,-0.034724,-0.512885,0.319018
1,-0.332568,-1.161011,0.890191,0.944020,0.801395
2,0.553347,0.346223,NaN,NaN,NaN


## 7. Reshape

In [153]:
# Create wide data
# Time surfing the internet
data = {
    'id': range(1, 4),
    'morning': [2, 2, 5],
    'afternoon': [3, 3, 4],
    'evening': [1, 0, 4]
}

df = pd.DataFrame(data)
df

,id,morning,afternoon,evening
0,1,2,3,1
1,2,2,3,0
2,3,5,4,4


### Wide to long

In [154]:
melted_df = df.melt(id_vars=["id"], var_name="time_of_day", value_name="surfing_hours")
melted_df

,id,time_of_day,surfing_hours
0,1,morning,2
1,2,morning,2
2,3,morning,5
3,1,afternoon,3
4,2,afternoon,3
5,3,afternoon,4
6,1,evening,1
7,2,evening,0
8,3,evening,4


### Long to wide

In [155]:
melted_df\
    .pivot(
        index="id", 
        columns="time_of_day", 
        values="surfing_hours"
    )\
    .reset_index()

time_of_day,id,afternoon,evening,morning
0,1,3,1,2
1,2,3,0,2
2,3,4,4,5
